### Algorithm:

1. Search the field replacing keywords in the URL

2. Get all the names and URLs from the page

3. Check if the keyword appears in each URL name

4. If the keyword appears in the URL name, add the name and URL to the list

5. Get 'next' URL and repeat the process until the keyword does not appear in the URL name

6. Get all the pages of the urls collected and get the
date filed for each URL

7. Sort the list by date filed

8. Remove the entries more than 30 days old

9. Print the list of names and URLs


In [219]:
import pandas as pd
import bs4
import requests
import concurrent.futures

In [120]:
names_and_links = []
keyword = "circuit"
header_link = 'https://search.sunbiz.org/'
r = requests.get(f'{header_link}Inquiry/CorporationSearch/SearchResults/EntityName/{keyword}/Page1?searchNameOrder={keyword.upper()}')
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [207]:
search_results = soup.find_all(attrs = {'id': 'search-results'})
nali = search_results[0].find_all('tr')
nali = [[nali[i].td.text, nali[i].a['href']] for i in range(1, len(nali))]

print(nali, len(nali), sep = '\n')

[['CIRCULAR 230 EDUCATIONAL FORUM, INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCULAR230EDUCATIONALFORUM%207678320&aggregateId=domnp-767832-ea02f603-064d-4f86-8b34-a8adf4b3a468&searchTerm=circuit&listNameOrder=CIRCULAR230EDUCATIONALFORUM%207678320'], ['CIRCULAR ADVERTISING & PROMOTIONS, INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCULARADVERTISINGPROMOTIONS%20P010000230591&aggregateId=domp-p01000023059-efaa5c23-f771-4d76-94a8-11b00476d65a&searchTerm=circuit&listNameOrder=CIRCULAR230EDUCATIONALFORUM%207678320'], ['CIRCULAR AUTO REPAIR CORPORATION', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=ForwardList&searchNameOrder=CIRCULARAUTOREPAIR%20F960360&aggregateId=domp-f96036-b77011f3-06fc-4c93-8dbb-4fd66ab76737&searchTerm=circuit&listNameOrder=CIRCULAR230EDUCATIONALFORUM%207678320'], ['CIRCULAR CLOCK

In [208]:
def check_name(keyword, namelink_list):
    length = len(namelink_list)
    status = []
    for idx in range(length):
        if keyword.lower() in namelink_list[idx][0].lower():
            status.append((namelink_list[idx][0], True))
        else:
            status.append((namelink_list[idx][0], False))
    return status
checked_list = check_name(keyword, nali)

In [209]:
checked_dt = pd.DataFrame(checked_list, columns = ['Name', 'Match'])
print(checked_dt)

                                                 Name  Match
0                CIRCULAR 230 EDUCATIONAL FORUM, INC.  False
1             CIRCULAR ADVERTISING & PROMOTIONS, INC.  False
2                    CIRCULAR AUTO REPAIR CORPORATION  False
3   CIRCULAR CLOCK DIAL WITH 12 CALIBRATION MARKS ...  False
4   CIRCULAR CLOCK DIAL WITH 12 CALIBRATION MARKS ...  False
5           CIRCULAR COMMERICAL CLEANING SERVICE, LLC  False
6                             CIRCULAR CONCEPTS, INC.  False
7                           CIRCULAR CONNECTORS, INC.  False
8                            CIRCULAR CONSULTING, LLC  False
9   CIRCULAR CONTEMPORARY HOMES DEVELOPMENT CORPOR...  False
10                        CIRCULAR CREATIVE GROUP LLC  False
11  CIRCULAR DESIGN OF WHITE WAVE BETWEEN BLUE & G...  False
12  CIRCULAR DESIGN WITH BANDS RADIATING OUT FROM ...  False
13  CIRCULAR DESIGN WITH OLD TELEPHONE IN MIDDLE S...  False
14  CIRCULAR DESIGN WITH SPECIFIC COLORS PLACEDWIT...  False
15  CIRCULAR DESIGN WITH

In [210]:
t_num = checked_dt[checked_dt['Match'] == True].count()
f_num = checked_dt[checked_dt['Match'] == False].count()
t_num[0]/(t_num[0] + f_num[0])

/tmp/ipykernel_13246/1113156225.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t_num[0]/(t_num[0] + f_num[0])


0.0

In [211]:
if (t_num[0]/(t_num[0] + f_num[0])) > 0.8:
    names_and_links.extend(nali)
    print('Next Page')
else:
    # names_and_links.extend(nali)
    # break statement
    print('Last Page')

print(names_and_links, len(names_and_links), sep = '\n')

Last Page
[['THE CIRCUIT LLC', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=CIRCUIT%20L060000400250&aggregateId=flal-l06000040025-a1a8ea44-65b9-40f2-9368-d9297c28ded9&searchTerm=circuit&listNameOrder=CIRCUIT%20L060000400250'], ['CIRCUIT COMPANY, INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=CIRCUIT%20M859840&aggregateId=domp-m85984-fedacf70-4d84-4ca7-bdca-3fb4aa2a8cb5&searchTerm=circuit&listNameOrder=CIRCUIT%20L060000400250'], ['THE CIRCUIT INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=CIRCUIT%20N513280&aggregateId=domnp-n51328-7f261b92-a411-4954-a86c-b46ca60083c6&searchTerm=circuit&listNameOrder=CIRCUIT%20L060000400250'], ['CIRCUIT, INC.', '/Inquiry/CorporationSearch/SearchResultDetail?inquirytype=EntityName&directionType=Initial&searchNameOrder=CIRCUIT%20P140000620120&aggregateId=domp-p

/tmp/ipykernel_13246/3509750301.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (t_num[0]/(t_num[0] + f_num[0])) > 0.8:


In [205]:
next_button = soup.find_all('div', attrs = {'class': 'navigationBarPaging'})
next_button_link = next_button[0].find_all('a')[-1]['href'].lstrip('/')
next_button_link

'Inquiry/CorporationSearch/SearchResults?InquiryType=EntityName&inquiryDirectionType=ForwardList&searchNameOrder=CIRCULAR%20P170000032080&SearchTerm=circuit&entityId=P17000003208&listNameOrder=CIRCUITTVRADIO%206694070'

In [206]:
r = requests.get(header_link + next_button_link)
r.raise_for_status()

soup = bs4.BeautifulSoup(r.text, 'html.parser')


print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PFV0FXXNRV">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-PFV0FXXNRV');
  </script>
  <script type="text/javascript">
   var _gaq = _gaq || [];
        _gaq.push(['_setAccount', 'UA-3263504-14']);
        _gaq.push(['_setDomainName', 'sunbiz.org']);
        _gaq.push(['_trackPageview']);

        (function () {
            var ga = document.createElement('script');
            ga.type = 'text/javascript';
            ga.async = true;
            ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
            var s = document.getElementsByTagName('script')[0];
            s.parentNode.insertBefore(ga, s);
        })();
  </script>
  <meta content="te

In [222]:
pd.DataFrame(names_and_links, columns= ['Name', 'Link'])

,Name,Link
0,THE CIRCUIT LLC,/Inquiry/CorporationSearch/SearchResultDetail?...
1,"CIRCUIT COMPANY, INC.",/Inquiry/CorporationSearch/SearchResultDetail?...
2,THE CIRCUIT INC.,/Inquiry/CorporationSearch/SearchResultDetail?...
3,"CIRCUIT, INC.",/Inquiry/CorporationSearch/SearchResultDetail?...
4,"CIRCUIT 14 HUMAN TRAFFICKING TASK FORCE, INC.",/Inquiry/CorporationSearch/SearchResultDetail?...
...,...,...
235,"CIRCUIT WORKS ELECTRONICS, LLC",/Inquiry/CorporationSearch/SearchResultDetail?...
236,CIRCUIT ZONE LTD LLC,/Inquiry/CorporationSearch/SearchResultDetail?...
237,"CIRCUL, INC.",/Inquiry/CorporationSearch/SearchResultDetail?...
238,"CIRCUL, INC.",/Inquiry/CorporationSearch/SearchResultDetail?...


In [223]:
def get_date_filed(name_and_link):
    r = requests.get(header_link + name_and_link[1])
    r.raise_for_status()
    comps = bs4.BeautifulSoup(r.text, 'html.parser')

    date_filed = comps.find_all('label', attrs = {'for': 'Detail_FileDate'})
    date_filed = date_filed[0].next_sibling.text
    name_and_link.append(date_filed)

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_date_filed, names_and_links)

https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
https://search.sunbiz.org/
h

In [234]:
namelink_dt = pd.DataFrame(names_and_links, columns = ['Name', 'Link', 'Date Filed'])
namelink_dt.count()

Name          240
Link          240
Date Filed    239
dtype: int64